In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train1-csv/train1.csv


In [20]:
import pandas as pd
import easyocr
import torch
import gc
from tqdm import tqdm
import os
import requests
from PIL import Image
from io import BytesIO

In [26]:
from tqdm import tqdm  # For progress bar

In [21]:
# Load your dataset
df = pd.read_csv('/kaggle/input/train1-csv/train1.csv')  # Replace with your dataset path
df['text_img'] = None  # Create the new column for storing extracted text

In [22]:
df.head()

,image_link,group_id,entity_name,entity_value,text_img
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram,None
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup,None
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram,None
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram,None
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram,None


In [25]:
# Initialize EasyOCR Reader, explicitly set gpu=True
reader = easyocr.Reader(['en'], gpu=True)

/opt/conda/lib/python3.10/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

In [23]:
# Define batch size (adjust based on your system capacity)
batch_size = 50

In [27]:
# Function to download and extract text from image
def extract_text_from_image(image_url):
    try:
        response = requests.get(image_url, timeout=10)  # Setting timeout for the request
        img = Image.open(BytesIO(response.content))
        text = reader.readtext(img, detail=0, paragraph=True)
        img.close()  # Close the image file to release memory
        del img  # Explicitly remove the image from memory
        return ' '.join(text)
    except Exception as e:
        print(f"Error processing image {image_url}: {e}")
        return None

In [28]:
# Process data in batches to avoid crashes
for i in range(0, len(df), batch_size):
    batch_df = df[i:i+batch_size]
    
    # Apply the extraction function with a progress bar for each batch
    tqdm.pandas(desc=f"Processing batch {i//batch_size + 1}/{len(df)//batch_size + 1}")
    batch_df['text_img'] = batch_df['image_link'].progress_apply(extract_text_from_image)
    
    # Store batch results to avoid losing data in case of a crash
    batch_df.to_csv(f'temp_results_batch_{i}.csv', index=False)
    
    # Merge the results back into the main dataframe
    df.loc[i:i+batch_size, 'text_img'] = batch_df['text_img']
    
    # Clear memory and GPU cache after each batch
    del batch_df
    torch.cuda.empty_cache()
    gc.collect()

# Final save to avoid data loss
df.to_csv('final_output_with_text.csv', index=False)

Processing batch 1/20:   0%|          | 0/50 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
Processing batch 1/20: 100%|██████████| 50/50 [01:27<00:00,  1.75s/it]
/tmp/ipykernel_36/3347625893.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['text_img'] = batch_df['image_link'].progress_apply(extract_text_from_image)
Processing batch 2/20:   0%|          | 0/50 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `

Error processing image https://m.media-amazon.com/images/I/1yw53vfQtS.jpg: cannot identify image file <_io.BytesIO object at 0x7bca283f3510>


Processing batch 11/20: 100%|██████████| 50/50 [01:04<00:00,  1.30s/it]
/tmp/ipykernel_36/3347625893.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['text_img'] = batch_df['image_link'].progress_apply(extract_text_from_image)
Processing batch 12/20:   0%|          | 0/50 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
Processing batch 12/20: 100%|██████████| 50/50 [01:18<00:00,  1.57s/it]
/tmp/ipykernel_36/3347625893.py:7: SettingWithCopyWarning: 
A value is trying to be

In [29]:
import shutil

# Move the file to the working directory for download
shutil.move('final_output_with_text.csv', '/kaggle/working/final_output_with_text.csv')


'/kaggle/working/final_output_with_text.csv'